In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)

In [2]:
df = pd.read_csv('./dataset/cars_train.csv', sep='\t', encoding='utf_16')

In [24]:
df.head()

,id,num_fotos,marca,modelo,versao,ano_de_fabricacao,ano_modelo,hodometro,cambio,num_portas,tipo,blindado,cor,tipo_vendedor,cidade_vendedor,estado_vendedor,anunciante,entrega_delivery,troca,elegivel_revisao,dono_aceita_troca,veiculo_único_dono,revisoes_concessionaria,ipva_pago,veiculo_licenciado,garantia_de_fábrica,revisoes_dentro_agenda,veiculo_alienado,preco
0,300716223898539419613863097469899222392,8.0,NISSAN,KICKS,1.6 16V FLEXSTART SL 4P XTRONIC,2017,2017.0,67772.0,CVT,4,Sedã,N,Branco,PF,Rio de Janeiro,São Paulo (SP),Pessoa Física,False,False,False,NaN,NaN,Todas as revisões feitas pela concessionária,IPVA pago,Licenciado,NaN,NaN,NaN,74732.590084
1,279639842134129588306469566150288644214,8.0,JEEP,COMPASS,2.0 16V FLEX LIMITED AUTOMÁTICO,2017,2017.0,62979.0,Automática,4,Sedã,N,Branco,PF,Belo Horizonte,Minas Gerais (MG),Pessoa Física,False,False,False,Aceita troca,NaN,NaN,IPVA pago,NaN,NaN,NaN,NaN,81965.332634
2,56414460810621048900295678236538171981,16.0,KIA,SORENTO,2.4 16V GASOLINA EX 7L AWD AUTOMÁTICO,2018,2019.0,44070.0,Automática,4,Sedã,N,Preto,PJ,Santos,São Paulo (SP),Loja,True,False,False,Aceita troca,NaN,NaN,NaN,NaN,NaN,NaN,NaN,162824.814472
3,56862509826849933428086372390159405545,14.0,VOLKSWAGEN,AMAROK,2.0 HIGHLINE 4X4 CD 16V TURBO INTERCOOLER DIES...,2013,2015.0,85357.0,Automática,4,Picape,N,Branco,PJ,Sorocaba,São Paulo (SP),Loja,True,True,False,Aceita troca,NaN,NaN,IPVA pago,Licenciado,NaN,NaN,NaN,123681.358857
4,338980975753200343894519909855598027197,8.0,SSANGYONG,KORANDO,2.0 GLS 4X4 16V TURBO DIESEL 4P AUTOMÁTICO,2013,2015.0,71491.0,Automática,4,Utilitário esportivo,N,Preto,PF,Rio de Janeiro,Rio de Janeiro (RJ),Pessoa Física,False,False,False,NaN,NaN,Todas as revisões feitas pela concessionária,NaN,NaN,Garantia de fábrica,Todas as revisões feitas pela agenda do carro,NaN,82419.763891


In [43]:
df[df.duplicated()]

,id,num_fotos,marca,modelo,versao,ano_de_fabricacao,ano_modelo,hodometro,cambio,num_portas,tipo,blindado,cor,tipo_vendedor,cidade_vendedor,estado_vendedor,anunciante,entrega_delivery,troca,elegivel_revisao,dono_aceita_troca,veiculo_único_dono,revisoes_concessionaria,ipva_pago,veiculo_licenciado,garantia_de_fábrica,revisoes_dentro_agenda,veiculo_alienado,preco


In [44]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29584 entries, 0 to 29583
Data columns (total 29 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id                       29584 non-null  object 
 1   num_fotos                29407 non-null  float64
 2   marca                    29584 non-null  object 
 3   modelo                   29584 non-null  object 
 4   versao                   29584 non-null  object 
 5   ano_de_fabricacao        29584 non-null  int64  
 6   ano_modelo               29584 non-null  float64
 7   hodometro                29584 non-null  float64
 8   cambio                   29584 non-null  object 
 9   num_portas               29584 non-null  int64  
 10  tipo                     29584 non-null  object 
 11  blindado                 29584 non-null  object 
 12  cor                      29584 non-null  object 
 13  tipo_vendedor            29584 non-null  object 
 14  cidade_vendedor       

In [28]:
df[['marca', 'modelo']].value_counts()

marca    modelo       
JEEP     COMPASS          1480
PEUGEOT  2008             1431
AUDI     Q3                914
FIAT     TORO              902
TOYOTA   COROLLA           867
                          ... 
BMW      X7                  1
         Z3                  1
PEUGEOT  RCZ                 1
CITROËN  XSARA PICASSO       1
DODGE    RAM                 1
Length: 457, dtype: int64

In [4]:
jeep = df[(df.marca == 'JEEP') & (df.modelo == 'COMPASS')][['marca', 'modelo', 'versao', 'tipo']]
jeep[~jeep.duplicated()].sort_values('versao')

,marca,modelo,versao,tipo
6214,JEEP,COMPASS,1.3 T270 TURBO FLEX LIMITED AT6,Utilitário esportivo
27741,JEEP,COMPASS,1.3 T270 TURBO FLEX LONGITUDE AT6,Sedã
8922,JEEP,COMPASS,1.3 T270 TURBO FLEX LONGITUDE AT6,Utilitário esportivo
8888,JEEP,COMPASS,1.3 T270 TURBO FLEX S AT6,Utilitário esportivo
18805,JEEP,COMPASS,1.3 T270 TURBO FLEX SPORT AT6,Sedã
1054,JEEP,COMPASS,2.0 16V DIESEL LIMITED 4X4 AUTOMÁTICO,Sedã
3889,JEEP,COMPASS,2.0 16V DIESEL LIMITED 4X4 AUTOMÁTICO,Utilitário esportivo
6170,JEEP,COMPASS,2.0 16V DIESEL LONGITUDE 4X4 AUTOMÁTICO,Utilitário esportivo
195,JEEP,COMPASS,2.0 16V DIESEL LONGITUDE 4X4 AUTOMÁTICO,Sedã
14466,JEEP,COMPASS,2.0 16V DIESEL NIGHT EAGLE 4X4 AUTOMÁTICO,Sedã


In [8]:
jeep.groupby(['marca', 'modelo', 'versao']).value_counts()

marca  modelo   versao                                     tipo                
JEEP   COMPASS  1.3 T270 TURBO FLEX LIMITED AT6            Utilitário esportivo       3
                1.3 T270 TURBO FLEX LONGITUDE AT6          Utilitário esportivo       4
                                                           Sedã                       1
                1.3 T270 TURBO FLEX S AT6                  Utilitário esportivo       2
                1.3 T270 TURBO FLEX SPORT AT6              Sedã                       2
                2.0 16V DIESEL LIMITED 4X4 AUTOMÁTICO      Sedã                      53
                                                           Utilitário esportivo      11
                2.0 16V DIESEL LONGITUDE 4X4 AUTOMÁTICO    Sedã                     133
                                                           Utilitário esportivo       2
                2.0 16V DIESEL NIGHT EAGLE 4X4 AUTOMÁTICO  Sedã                       3
                2.0 16V DIESEL S 4X4 AUT

In [9]:
df.value_counts('tipo')

tipo
Sedã                    16429
Hatchback                4924
Picape                   4849
Utilitário esportivo     3322
Perua/SW                   27
Cupê                       26
Minivan                     7
dtype: int64

In [22]:
df[(df.marca == 'JEEP') & (df.modelo == 'COMPASS')].groupby(['versao', 'tipo']).preco.mean()

versao                                     tipo                
1.3 T270 TURBO FLEX LIMITED AT6            Utilitário esportivo    221165.304662
1.3 T270 TURBO FLEX LONGITUDE AT6          Sedã                    147902.674137
                                           Utilitário esportivo    186657.981761
1.3 T270 TURBO FLEX S AT6                  Utilitário esportivo    273813.874375
1.3 T270 TURBO FLEX SPORT AT6              Sedã                    146784.708381
2.0 16V DIESEL LIMITED 4X4 AUTOMÁTICO      Sedã                    194626.092402
                                           Utilitário esportivo    231870.382610
2.0 16V DIESEL LONGITUDE 4X4 AUTOMÁTICO    Sedã                    157798.475429
                                           Utilitário esportivo    274469.203173
2.0 16V DIESEL NIGHT EAGLE 4X4 AUTOMÁTICO  Sedã                    127544.469034
2.0 16V DIESEL S 4X4 AUTOMÁTICO            Sedã                    185830.224403
                                           Ut

In [58]:
# Sugestão: fazer feature engineering para determinar se o carro é flex, gasolina, diesel, eletrico, etc
# Sugestão 2: cilindradas do motor (1.0, etc.)

for versao in df.versao.sort_values().unique():
    print(versao)

0.6 HÍBRIDO REX FULL 4P AUTOMÁTICO
1.0 12V FLEX 4P MANUAL
1.0 12V FLEX EVOLUTION MANUAL
1.0 12V FLEX SENSE MANUAL
1.0 12V FLEX V-DRIVE MANUAL
1.0 12V FLEX VISION MANUAL
1.0 12V MPI TOTALFLEX 4P MANUAL
1.0 12V MPI TOTALFLEX TRENDLINE 4P MANUAL
1.0 12V SCE FLEX AUTHENTIQUE MANUAL
1.0 12V SCE FLEX EXPRESSION MANUAL
1.0 12V SCE FLEX GT LINE MANUAL
1.0 12V SCE FLEX INTENSE MANUAL
1.0 12V SCE FLEX LIFE MANUAL
1.0 12V SCE FLEX OUTSIDER MANUAL
1.0 12V SCE FLEX S EDITION MANUAL
1.0 12V SCE FLEX VIBE MANUAL
1.0 12V SCE FLEX ZEN MANUAL
1.0 16V FLEX 2P MANUAL
1.0 16V FLEX 4P MANUAL
1.0 170 TSI TOTAL FLEX CONNECT 4P MANUAL
1.0 170 TSI TOTAL FLEX XTREME 4P MANUAL
1.0 200 TSI COMFORTLINE AUTOMÁTICO
1.0 200 TSI HIGHLINE AUTOMÁTICO
1.0 200 TSI SENSE AUTOMÁTICO
1.0 200 TSI TOTAL FLEX AUTOMÁTICO
1.0 200 TSI TOTAL FLEX COMFORTLINE AUTOMÁTICO
1.0 200 TSI TOTAL FLEX COMFORTLINE TIPTRONIC
1.0 200 TSI TOTAL FLEX HIGHLINE AUTOMÁTICO
1.0 200 TSI TOTAL FLEX MANUAL
1.0 200 TSI TOTAL FLEX SENSE AUTOMÁTICO
1.0 5 AN

In [27]:
df[~df.versao.str.contains('GASOL|GAS|FLEX|DIESEL|ELÉT|ELET|ELEC|HíB|HYB|CHARG|TETRAFUEL', case=False)] \
    .loc[:, ['marca', 'modelo', 'versao']] \
    .sort_values('versao')\
    .drop_duplicates() \
    .apply(lambda row: print(f'Marca {row.marca} | modelo {row.modelo} | versao {row.versao}'), axis=1) 
    # print(versao)

Marca VOLKSWAGEN | modelo POLO | versao 1.0 200 TSI COMFORTLINE AUTOMÁTICO
Marca VOLKSWAGEN | modelo VIRTUS | versao 1.0 200 TSI COMFORTLINE AUTOMÁTICO
Marca VOLKSWAGEN | modelo POLO | versao 1.0 200 TSI HIGHLINE AUTOMÁTICO
Marca VOLKSWAGEN | modelo VIRTUS | versao 1.0 200 TSI HIGHLINE AUTOMÁTICO
Marca VOLKSWAGEN | modelo POLO | versao 1.0 200 TSI SENSE AUTOMÁTICO
Marca VOLKSWAGEN | modelo POLO | versao 1.4 250 TSI GTS AUTOMÁTICO
Marca VOLKSWAGEN | modelo VIRTUS | versao 1.4 250 TSI GTS AUTOMÁTICO
Marca JAGUAR | modelo F-PACE | versao 2.0 16V INGENIUM PRESTIGE AWD 4P AUTOMÁTICO
Marca AUDI | modelo RS7 | versao 4.0 SPORTBACK V8 TWINTURBO MHEV TIPTRONIC
Marca BMW | modelo M5 | versao 4.4 V8 TWIN POWER M XDRIVE STEPTRONIC


17       None
13924    None
24440    None
28697    None
23631    None
13118    None
733      None
16028    None
23217    None
23307    None
dtype: object

In [36]:
df[(df.versao.str.contains('GAS', case=False)) ] \
    .loc[:, ['marca', 'modelo', 'versao']] \
    .sort_values('versao')\
    .drop_duplicates() \
    .apply(lambda row: print(f'Marca {row.marca} | modelo {row.modelo} | versao {row.versao}'), axis=1) 
    # print(versao)

Marca EFFA | modelo PICAPE CAÇAMBA LONGA | versao 1.0 8V GASOLINA 2P MANUAL
Marca VOLKSWAGEN | modelo GOL | versao 1.0 8V GASOLINA 2P MANUAL
Marca SMART | modelo FORTWO | versao 1.0 BRABUS COUPÊ 3 CILINDROS 12V TURBO GASOLINA 2P AUTOMÁTIZADO
Marca SMART | modelo FORTWO | versao 1.0 CABRIO TURBO 12V GASOLINA 2P AUTOMÁTICO
Marca SMART | modelo FORTWO | versao 1.0 COUPÉ 3 CILINDROS 12V GASOLINA 2P AUTOMÁTICO
Marca SMART | modelo FORTWO | versao 1.0 COUPÊ TURBO 12V GASOLINA 2P AUTOMÁTICO
Marca FORD | modelo FIESTA | versao 1.0 ECOBOOST TITANIUM PLUS HATCH 12V GASOLINA 4P POWERSHIFT
Marca KIA | modelo PICANTO | versao 1.0 EX 12V GASOLINA 4P AUTOMÁTICO
Marca SUZUKI | modelo JIMNY | versao 1.3 4ALL 4X4 16V GASOLINA 2P MANUAL
Marca SUZUKI | modelo JIMNY | versao 1.3 4SPORT 4X4 16V GASOLINA 2P MANUAL
Marca SUZUKI | modelo JIMNY | versao 1.3 4SPORT FOREST 4X4 16V GASOLINA 2P MANUAL
Marca MERCEDES-BENZ | modelo GLB 200 | versao 1.3 CGI GASOLINA ADVANCE 7G-DCT
Marca MERCEDES-BENZ | modelo A 200 | 

5168     None
24248    None
8925     None
4679     None
12938    None
         ... 
19012    None
21417    None
9657     None
3891     None
13022    None
Length: 822, dtype: object

In [25]:
df.groupby('tipo').preco.describe().T

tipo,Cupê,Hatchback,Minivan,Perua/SW,Picape,Sedã,Utilitário esportivo
count,2.600000e+01,4924.000000,7.000000,27.000000,4.849000e+03,16429.000000,3.322000e+03
mean,4.132604e+05,92955.616973,65217.048192,110059.507733,1.589062e+05,118191.936375,2.261230e+05
std,1.872609e+05,55304.899285,22929.308025,138245.325557,1.014310e+05,58672.726957,9.307469e+04
min,1.417922e+05,13576.401422,40500.340767,18378.042702,1.364289e+04,9869.950645,3.773681e+04
25%,3.101223e+05,56579.593842,42488.368551,39797.160187,8.521248e+04,77885.302468,1.625234e+05
50%,3.649848e+05,78858.785041,70327.081458,61022.657027,1.398146e+05,108016.327220,2.169202e+05
75%,4.757234e+05,115383.512522,83795.948202,107051.633983,2.092586e+05,145409.379808,2.732660e+05
max,1.019841e+06,676234.221089,93123.281617,660950.461474,1.359813e+06,711460.364417,1.349748e+06


In [40]:
df.groupby('marca').preco.median().sort_values()

marca
EFFA              40163.175829
BRM               47762.671073
CHRYSLER          57530.513198
CITROËN           61708.494392
SMART             66929.542662
ALFA ROMEO        69926.750037
SSANGYONG         73173.086612
RENAULT           75871.664317
DODGE             76463.935136
HYUNDAI           77118.762163
CHEVROLET         79934.798236
JAC               80714.531749
LIFAN             84637.784352
SUZUKI            85950.328726
HONDA             91506.027179
NISSAN            91691.281136
FIAT              93202.070187
VOLKSWAGEN       103350.092101
FORD             103619.715970
CHERY            113645.340606
FERRARI          114439.236279
SUBARU           115592.280402
MITSUBISHI       117222.069832
KIA              118306.655455
PEUGEOT          126420.198191
TOYOTA           129875.164942
MINI             132806.879317
JEEP             133126.835734
TROLLER          142571.038905
AUDI             144563.336793
IVECO            151197.557227
MERCEDES-BENZ    158628.067130
BM

In [41]:
df.value_counts('cidade_vendedor')

cidade_vendedor
São Paulo              6158
Rio de Janeiro         5793
Belo Horizonte         2172
Presidente Prudente    1536
Goiânia                1396
                       ... 
Paranaiguara              1
Paracambi                 1
Panambi                   1
Palmital                  1
Luiz Alves                1
Length: 575, dtype: int64